In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from PIL import Image

In [2]:
article_tags = pd.read_csv("articles_tags.csv")
article_text = pd.read_csv("../resources/articles_text.csv")
article_tags= article_tags.set_index("Resource URL")
article_text = article_text.set_index("Resource URL")
article_text = article_text[~article_text.index.duplicated(keep='first')]

In [3]:
for i in article_text.index:
    article_text.loc[i, "tech_tag"] =  article_tags.loc[i]["Technical (Y/N)"][0]

In [4]:
data = (article_text[(article_text["tech_tag"]=="Y") | (article_text["tech_tag"]=="N")])
data_src = data
data_src = data_src[["Resource Title", "text", "tech_tag"]]
data_src.loc[:, "text"] =  data_src["Resource Title"] + " " +data_src["text"]
data_src.loc[:, "tech_tag"] = data_src["tech_tag"].map({"N": 0, "Y":1})
data_src.drop("Resource Title", axis=1, inplace=True)
data_src = data_src.dropna().reset_index(drop=True)
data=data_src

/opt/conda/lib/python3.8/site-packages/pandas/core/indexing.py:1783: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item_labels[indexer[info_axis]]] = value
/opt/conda/lib/python3.8/site-packages/pandas/core/frame.py:4162: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [5]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(data['text'], data['tech_tag'], test_size = 0.5, random_state = 0)

In [6]:
print('rows in test set: ' + str(x_test.shape))
print('rows in train set: ' + str(x_train.shape))

rows in test set: (500,)
rows in train set: (500,)


In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer

In [8]:
lst = x_train.tolist()
vectorizer = TfidfVectorizer(
    input= lst ,  # input is the actual text
    lowercase=True,      # convert to lowercase before tokenizing
    stop_words='english' # remove stop words
)
features_train_transformed = vectorizer.fit_transform(x_train) #gives tf idf vector for x_train
features_test_transformed  = vectorizer.transform(x_test) #gives tf idf vector for x_test

In [10]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

In [89]:
classifiers = [
    KNeighborsClassifier(3),
    SVC(kernel="linear", C=0.025),
    SVC(gamma=2, C=1),
    GaussianProcessClassifier(1.0 * RBF(1.0)),
    DecisionTreeClassifier(max_depth=5),
    RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1),
    MLPClassifier(alpha=1, max_iter=1000),
    AdaBoostClassifier(),
    GaussianNB(),
    QuadraticDiscriminantAnalysis()]

names = ["Nearest Neighbors", "Linear SVM", "RBF SVM", "Gaussian Process",
         "Decision Tree", "Random Forest", "Neural Net", "AdaBoost",
         "Naive Bayes", "QDA"]


for name, classifier in zip(names, classifiers):
    try:
        classifier.fit(features_train_transformed, y_train)
        print(name+" accuracy {:.2f}%".format(classifier.score(features_test_transformed, y_test) * 100))
    except:
        continue
    

Nearest Neighbors accuracy 89.60%
Linear SVM accuracy 49.40%
RBF SVM accuracy 90.20%
Decision Tree accuracy 80.60%
Random Forest accuracy 56.80%
Neural Net accuracy 91.40%
AdaBoost accuracy 89.80%


In [36]:
classifiers = [

    MLPClassifier(alpha=1, hidden_layer_sizes=20, max_iter=1000),
]

names = [ "Neural Net"]


for name, classifier in zip(names, classifiers):
    try:
        classifier.fit(features_train_transformed, y_train)
        print(name+" accuracy {:.2f}%".format(classifier.score(features_test_transformed, y_test) * 100))
    except:
        continue
    

Neural Net accuracy 91.40%


In [46]:
hash_map.invert()

AttributeError: 'dict' object has no attribute 'invert'

In [47]:
hash_map = {'Y': 1, 'N': 0}
article_text["NN_Pred"]=""
for i in article_text.index:
    if article_text.loc[i].tech_tag not in hash_map.keys():
        continue
    target = hash_map[article_text.loc[i].tech_tag]
    text = article_text.loc[i].text
    if type(text) != type(' '):
        continue
    text = vectorizer.transform([text])
    pred = classifier.predict(text)[0]
    article_text.loc[i,"NN_Pred"]="Y" if pred else "N"
    if pred!=target:
        print(i, "Predicted: ",pred, " Actual: ", target)
    

https://jamanetwork.com/journals/jama/pages/coronavirus-alert Predicted:  0  Actual:  1
https://jamanetwork.com/collections/5950/vaccination Predicted:  0  Actual:  1
https://jamanetwork.com/journals/jama/fullarticle/2779817 Predicted:  0  Actual:  1
https://jamanetwork.com/journals/jamaneurology/fullarticle/2779916 Predicted:  0  Actual:  1
https://www.nejm.org/pulmonary?topic=12&q=*&page=5 Predicted:  0  Actual:  1
https://www.nature.com/articles/d41586-021-01251-0 Predicted:  1  Actual:  0
https://www.nature.com/articles/d41586-021-01277-4 Predicted:  0  Actual:  1
https://www.nature.com/articles/s41574-021-00509-x Predicted:  1  Actual:  0
https://www.nature.com/articles/s41556-021-00685-y Predicted:  1  Actual:  0
https://jamanetwork.com/journals/jamanetworkopen/fullarticle/2779869 Predicted:  0  Actual:  1
https://www.nejm.org/doi/full/10.1056/NEJMc2106004 Predicted:  1  Actual:  0
https://www.nejm.org/coronavirus Predicted:  1  Actual:  0
https://www.nature.com/articles/s41467-0

In [49]:
article_text.to_csv("nn_pred.csv")

In [17]:
    text = vectorizer.transform([article_text.loc[i].text])

In [23]:
result[0]

0

classifier accuracy 91.00%
